In [ ]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

onnxModelPath = '/data2/user/sjj995/TensorRT/testing/yolov3-12.onnx'
tensorrt_file_name = '/data2/user/sjj995/TensorRT/testing/tiny-yolov3-11.trt'

# create builder <- optimization profile, Takes a network in TensorRT and generates an engine that is optimized for the target platform. 
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)
trt.init_libnvinfer_plugins(None,'')

In [ ]:
trt_runtime

In [ ]:
# trt(engine) model file version = 8.2.4 / tensorrt version = 8.4.1 <-  이렇게 버전 안 맞을때는 안됨
# 버전 8.4.1 로 trtexec 다시 해서 모델 만들어서 하면 됨!

with open(tensorrt_file_name, 'rb') as f:
    engine_data = f.read()

engine = engine_data
engine = trt_runtime.deserialize_cuda_engine(engine_data)

In [ ]:
for binding in engine:
    print(binding,engine.get_binding_shape(binding))

In [ ]:
output_data = ['yolonms_layer_1:2','yolonms_layer_1:1','yolonms_layer_1']

In [ ]:
for binding in engine:
    if binding in output_data:
        print(binding)
        print(engine.get_binding_shape(binding))
    else:
        print(binding)
        print(engine.get_binding_shape(binding))

In [ ]:
engine.num_bindings

In [ ]:
class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()

In [ ]:
inputs = []
outputs = []
bindings = []
stream = cuda.Stream()

for binding in engine:
    print(engine.get_binding_shape(binding))
    print(engine.max_batch_size)
    size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
    print(size)
    dtype = trt.nptype(engine.get_binding_dtype(binding)) #float 32 안됨
    print(dtype)
    host_mem = cuda.pagelocked_empty(size, dtype)
    print(len(host_mem))
    print(type(host_mem))
    device_mem = cuda.mem_alloc(host_mem.nbytes)
    print(int(device_mem))
    bindings.append(int(device_mem))
    # Append to the appropriate list.
    if engine.binding_is_input(binding):
        inputs.append(HostDeviceMem(host_mem, device_mem))
    else:
        outputs.append(HostDeviceMem(host_mem, device_mem))

In [ ]:
len(inputs)

In [ ]:
import numpy as np

for i in range(len(inputs)):
    print(inputs[i])

In [ ]:
len(outputs)

In [ ]:
outputs

In [ ]:
for output in outputs:
    print(output.host)
    print(len(output.host))

In [ ]:
outputs

In [ ]:
bindings

In [ ]:
stream

In [ ]:
context = engine.create_execution_context()

In [ ]:
context

In [ ]:
import torch
import numpy as np
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


input_data = torch.randn(1,3, 224, 224).to(device)
input_data = to_numpy(input_data)

In [ ]:
input_data

In [ ]:
def do_inference(context, bindings, inputs, outputs, stream, batch_size=1):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async(batch_size=batch_size, bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

In [ ]:
# This function is generalized for multiple inputs/outputs for full dimension networks.
# inputs and outputs are expected to be lists of HostDeviceMem objects.
def do_inference_v2(context, bindings, inputs, outputs, stream):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

In [ ]:
hosts = [input.host for input in inputs]
trt_types = [trt.int32, trt.int32, trt.int32]

In [ ]:
for input in inputs:
    print(input)
    print(type(input))
    print(input.device)
    print(len(input.host))
    print(type(input.host))
    np.copyto(input.host,np.ravel(input_data))

In [ ]:
context

In [ ]:
bindings

In [ ]:
trt_outputs = do_inference_v2(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream)

In [ ]:
# 싱글 thread 에서 처리해야함. https://forums.developer.nvidia.com/t/tensorrt-do-inference-error/77055/3

import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit # https://forums.developer.nvidia.com/t/logicerror-explicit-context-dependent-failed-invalid-device-context-no-currently-active-context/64149
import torch
import numpy as np

#onnxModelPath = '/datadrive/tensorrt/test_onnx'
tensorrt_file_name = '/data2/user/sjj995/TensorRT/testing/yolov3-12.trt'
#tensorrt_file_name = '/data2/user/sjj995/TensorRT/trt_models/fcn-resnet50-12.trt'

# create builder <- optimization profile, Takes a network in TensorRT and generates an engine that is optimized for the target platform. 
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)
trt.init_libnvinfer_plugins(None,'')

EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

with open(tensorrt_file_name, 'rb') as f:
    engine_data = f.read()
engine = trt_runtime.deserialize_cuda_engine(engine_data)
print(engine)

class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()
    
inputs = []
outputs = []
bindings = []
stream = cuda.Stream()
context = engine.create_execution_context()

for binding in engine:
    print(binding)
    size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size

    dtype = trt.nptype(engine.get_binding_dtype(binding)) #float 32 안됨

    host_mem = cuda.pagelocked_empty(size, dtype)

    device_mem = cuda.mem_alloc(host_mem.nbytes)

    bindings.append(int(device_mem))
    # Append to the appropriate list.
    if engine.binding_is_input(binding):
        inputs.append(HostDeviceMem(host_mem, device_mem))
    else:
        outputs.append(HostDeviceMem(host_mem, device_mem))
       
# for inp in inputs:
#     print(inp.host.shape)

# for oup in outputs:
#     print(oup.host.shape)
#     print(oup.device)
#     print(stream)

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


input_data = torch.randn(1,3, 1, 1).to(device)
input_data = to_numpy(input_data)

for input in inputs:
    np.copyto(input.host,np.ravel(input_data))

# This function is generalized for multiple inputs/outputs for full dimension networks.
# inputs and outputs are expected to be lists of HostDeviceMem objects.
def do_inference_v2(context, bindings, inputs, outputs, stream):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

trt_outputs = do_inference_v2(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream)

In [ ]:
trt_outputs[1].shape

In [ ]:
trt_outputs[0].shape

In [ ]:
stream = cuda.Stream()
stream.ptr

In [ ]:
trt_outputs

In [ ]:
trt_outputs

In [ ]:
# 싱글 thread 에서 처리해야함. https://forums.developer.nvidia.com/t/tensorrt-do-inference-error/77055/3

import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit # https://forums.developer.nvidia.com/t/logicerror-explicit-context-dependent-failed-invalid-device-context-no-currently-active-context/64149
import torch
import numpy as np

#onnxModelPath = '/datadrive/tensorrt/onnx_models/mobilenetv2-7.onnx'
tensorrt_file_name = '/datadrive/tensorrt/mobilenet.trt'

# create builder <- optimization profile, Takes a network in TensorRT and generates an engine that is optimized for the target platform. 
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)

EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

with open(tensorrt_file_name, 'rb') as f:
    engine_data = f.read()
engine = trt_runtime.deserialize_cuda_engine(engine_data)


class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()
    
inputs = []
outputs = []
bindings = []
stream = cuda.Stream()
context = engine.create_execution_context()

for binding in engine:
    print(engine.get_binding_shape(binding))
    print(engine.max_batch_size)
    size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
    print(size)
    dtype = trt.nptype(engine.get_binding_dtype(binding)) #float 32 안됨
    print(dtype)
    host_mem = cuda.pagelocked_empty(size, dtype)
    print(len(host_mem))
    device_mem = cuda.mem_alloc(host_mem.nbytes)
    print(int(device_mem))
    bindings.append(int(device_mem))
    # Append to the appropriate list.
    if engine.binding_is_input(binding):
        inputs.append(HostDeviceMem(host_mem, device_mem))
    else:
        outputs.append(HostDeviceMem(host_mem, device_mem))
       

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


input_data = torch.randn(1,3, 224, 224).to(device)
input_data = to_numpy(input_data)

for input in inputs:
    np.copyto(input.host,np.ravel(input_data))

# This function is generalized for multiple inputs/outputs for full dimension networks.
# inputs and outputs are expected to be lists of HostDeviceMem objects.
def do_inference_v2(context, bindings, inputs, outputs, stream):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

trt_outputs = do_inference_v2(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream)

In [31]:
# 싱글 thread 에서 처리해야함. https://forums.developer.nvidia.com/t/tensorrt-do-inference-error/77055/3
import time
import onnxruntime
import numpy as np
from PIL import Image
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit # https://forums.developer.nvidia.com/t/logicerror-explicit-context-dependent-failed-invalid-device-context-no-currently-active-context/64149
import torch
import numpy as np

#onnxModelPath = '/datadrive/tensorrt/onnx_models/mobilenetv2-7.onnx'
tensorrt_file_name = '/data2/user/sjj995/TensorRT/testing/jupyter_yolo.trt'

# create builder <- optimization profile, Takes a network in TensorRT and generates an engine that is optimized for the target platform. 
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)
trt.init_libnvinfer_plugins(None,'')
    

EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

with open(tensorrt_file_name, 'rb') as f:
    engine_data = f.read()
engine = trt_runtime.deserialize_cuda_engine(engine_data)


class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()
    
inputs = []
outputs = []
bindings = []
output_data = ['yolonms_layer_1/ExpandDims_3:0','yolonms_layer_1/ExpandDims_1:0','yolonms_layer_1/concat_2:0']
output_shape = []
stream = cuda.Stream()
context = engine.create_execution_context()

for binding in engine:
    print(binding)
    if binding in output_data:
        output_shape.append(engine.get_binding_shape(binding))
    print(engine.get_binding_shape(binding))
    #print(engine.max_batch_size)
    size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
    #print(size)
    dtype = trt.nptype(engine.get_binding_dtype(binding)) #float 32 안됨
    #print(dtype)
    host_mem = cuda.pagelocked_empty(size, dtype)
    #print(len(host_mem))
    device_mem = cuda.mem_alloc(host_mem.nbytes)
    #print(int(device_mem))
    bindings.append(int(device_mem))
    # Append to the appropriate list.
    if engine.binding_is_input(binding):
        inputs.append(HostDeviceMem(host_mem, device_mem))
    else:
        outputs.append(HostDeviceMem(host_mem, device_mem))
       

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def letterbox_image(image, size):
    iw, ih = image.size
    w, h = size
    scale = min(w/iw, h/ih)
    nw = int(iw*scale)
    nh = int(ih*scale)

    image = image.resize((nw,nh), Image.BICUBIC)
    new_image = Image.new('RGB', size, (128,128,128))
    new_image.paste(image, ((w-nw)//2, (h-nh)//2))
    return new_image

def preprocess(img):
    model_image_size = (416, 416)
    boxed_image = letterbox_image(img, tuple(reversed(model_image_size)))
    image_data = np.array(boxed_image, dtype='float32')
    image_data /= 255.
    image_data = np.transpose(image_data, [2, 0, 1])
    image_data = np.expand_dims(image_data, 0)
    return image_data

image = Image.open("/data2/user/sjj995/TensorRT/person.jpg")
image_data = preprocess(image)
image_size = np.array([image.size[1], image.size[0]], dtype=np.float32).reshape(1, 2)
input_data = [image_data,image_size]

for i in range(len(inputs)):
    np.copyto(inputs[i].host,np.ravel(input_data[i]))

# This function is generalized for multiple inputs/outputs for full dimension networks.
# inputs and outputs are expected to be lists of HostDeviceMem objects.
def do_inference_v2(context, bindings, inputs, outputs, stream):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

trt_outputs = do_inference_v2(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream)



trt_outputs = [output.reshape(shape) for output, shape in zip(trt_outputs, output_shape)]
scores, boxes, indices = trt_outputs
out_boxes, out_scores, out_classes = [], [], []
for idx_ in indices:
    out_classes.append(idx_[1])
    out_scores.append(scores[tuple(idx_)])
    idx_1 = (idx_[0], idx_[2])
    out_boxes.append(boxes[idx_1])

label =["person","bicycle","car","motorbike","aeroplane","bus","train","truck","boat",
    "traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat",
    "dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack","umbrella",
    "handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite","baseball bat",
    "baseball glove","skateboard","surfboard","tennis racket","bottle","wine glass","cup","fork",
    "knife","spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot","hot dog",
    "pizza","donut","cake","chair","sofa","pottedplant","bed","diningtable","toilet","tvmonitor",
    "laptop","mouse","remote","keyboard","cell phone","microwave","oven","toaster","sink",
    "refrigerator","book","clock","vase","scissors","teddy bear","hair drier","toothbrush"]

out_boxes = np.array(out_boxes).tolist()
out_scores = np.array(out_scores).tolist()
out_classes = np.array(out_classes).tolist()


for i, c in reversed(list(enumerate(out_classes))):
    print("box:", out_boxes[i])
    print("score:", out_scores[i],",", label[c])

[07/15/2022-08:47:19] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0
[07/15/2022-08:47:19] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0
input_1
(1, 3, 416, 416)
image_shape
(1, 2)
yolonms_layer_1/ExpandDims_3:0
(1, 80, 10647)
yolonms_layer_1/ExpandDims_1:0
(1, 10647, 4)
yolonms_layer_1/concat_2:0
(20, 3)
[07/15/2022-08:47:19] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[07/15/2022-08:47:19] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[07/15/2022-08:47:19] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function wi

/tmp/ipykernel_1103683/253886674.py:53: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
/tmp/ipykernel_1103683/253886674.py:80: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  image = image.resize((nw,nh), Image.BICUBIC)


In [37]:

trt_outputs.sort(key=lambda x: priority_dict[x])

NameError: name 'priority_dict' is not defined

In [67]:
priority_dict = {1:0,0:1}

In [58]:
trt_outputs[v for v in priority_dict.values()]

SyntaxError: invalid syntax (2096033498.py, line 1)

In [60]:
[v for v in priority_dict.values()]

[1, 0]

In [62]:
for v in priority_dict.values():
    print(trt_outputs[v])

[[[-365.5753   -67.90302  -97.76094  173.55362]
  [-601.63525  -76.2155   171.91776  186.59631]
  [-666.9932  -582.22943  198.83447  711.8014 ]
  ...
  [ 893.7547  1166.1498   925.33185 1175.7616 ]
  [ 873.9263  1156.6022   939.07385 1180.2361 ]
  [ 901.84143 1098.2267   925.2407  1237.4456 ]]]
[[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 6.4237859e-10
   0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.7943142e-08
   0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.1111697e-08
   0.0000000e+00 0.0000000e+00]
  ...
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.2434498e-13
   0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 6.5902839e-13
   0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.2329694e-13
   0.0000000e+00 0.0000000e+00]]]


In [66]:
trt_outputs.sort(key=lambda x: priority_dict[x])

TypeError: unhashable type: 'numpy.ndarray'

In [46]:
type(trt_outputs[0])

numpy.ndarray

In [74]:
import json
import pandas as pd
origin_json_path = '../report1.json'
with open(origin_json_path,'r') as f:
    origin_data = json.load(f)

final_data = dict(origin_data)
final_pd = pd.DataFrame(final_data).transpose()
final_pd.columns=['Input Name','Input Shape','Output Name','Output Shape','abs Similarity','rel Similarity','Max Diff','Max Diff Pos']
print('[[Final DataFrame]]')
print(final_pd)

[[Final DataFrame]]
                                         Input Name          Input Shape  \
ResNet101-DUC-12                             [data]  ([1, 3, 800, 800],)   
age_googlenet                               [input]  ([1, 3, 224, 224],)   
arcfaceresnet100-8                           [data]  ([1, 3, 112, 112],)   
emotion-ferplus-8                          [Input3]    ([1, 1, 64, 64],)   
fcn-resnet50-12                             [input]  ([1, 3, 416, 416],)   
gender_googlenet                            [input]  ([1, 3, 224, 224],)   
retinanet-9                                 [input]  ([1, 3, 480, 640],)   
version-RFB-320                             [input]  ([1, 3, 240, 320],)   
version-RFB-640                             [input]  ([1, 3, 480, 640],)   
vgg_ilsvrc_16_age_chalearn_iccv2015         [input]  ([1, 3, 224, 224],)   
vgg_ilsvrc_16_age_imdb_wiki                 [input]  ([1, 3, 224, 224],)   
vgg_ilsvrc_16_gender_imdb_wiki              [input]  ([1, 3, 224, 22